In [1]:
from keras.layers import Conv2D, Input,MaxPool2D, Reshape,Activation,Flatten, Dense, Permute
from keras.models import Model, Sequential
import tensorflow as tf
from keras.layers.advanced_activations import PReLU
import numpy as np
from keras.utils import plot_model

Using TensorFlow backend.


In [2]:
def create_Kao_Onet( weight_path = 'models/48net.h5'):
    input = Input(shape = [48,48,3])
    x = Conv2D(32, (3, 3), strides=1, padding='valid', name='conv1')(input)
    x = PReLU(shared_axes=[1,2],name='prelu1')(x)
    x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)
    x = Conv2D(64, (3, 3), strides=1, padding='valid', name='conv2')(x)
    x = PReLU(shared_axes=[1,2],name='prelu2')(x)
    x = MaxPool2D(pool_size=3, strides=2)(x)
    x = Conv2D(64, (3, 3), strides=1, padding='valid', name='conv3')(x)
    x = PReLU(shared_axes=[1,2],name='prelu3')(x)
    x = MaxPool2D(pool_size=2)(x)
    x = Conv2D(128, (2, 2), strides=1, padding='valid', name='conv4')(x)
    x = PReLU(shared_axes=[1,2],name='prelu4')(x)
    x = Permute((3,2,1))(x)
    x = Flatten()(x)
    x = Dense(256, name='conv5') (x)
    x = PReLU(name='prelu5')(x)

    classifier = Dense(2, activation='softmax',name='conv6-1')(x)
    bbox_regress = Dense(4,name='conv6-2')(x)
    landmark_regress = Dense(10,name='conv6-3')(x)
    model = Model([input], [classifier, bbox_regress, landmark_regress])
    model.load_weights(weight_path, by_name=True)

    return model

In [3]:
def create_Kao_Rnet (weight_path = 'models/24net.h5'):
    input = Input(shape=[24, 24, 3])  # change this shape to [None,None,3] to enable arbitraty shape input
    x = Conv2D(28, (3, 3), strides=1, padding='valid', name='conv1')(input)
    x = PReLU(shared_axes=[1, 2], name='prelu1')(x)
    x = MaxPool2D(pool_size=3,strides=2, padding='same')(x)

    x = Conv2D(48, (3, 3), strides=1, padding='valid', name='conv2')(x)
    x = PReLU(shared_axes=[1, 2], name='prelu2')(x)
    x = MaxPool2D(pool_size=3, strides=2)(x)

    x = Conv2D(64, (2, 2), strides=1, padding='valid', name='conv3')(x)
    x = PReLU(shared_axes=[1, 2], name='prelu3')(x)
    x = Permute((3, 2, 1))(x)
    x = Flatten()(x)
    x = Dense(128, name='conv4')(x)
    x = PReLU( name='prelu4')(x)
    classifier = Dense(2, activation='softmax', name='conv5-1')(x)
    bbox_regress = Dense(4, name='conv5-2')(x)
    model = Model([input], [classifier, bbox_regress])
    model.load_weights(weight_path, by_name=True)
    return model

In [4]:
def create_Kao_Pnet( weight_path = 'models/12net.h5'):
    input = Input(shape=[None, None, 3])
    x = Conv2D(10, (3, 3), strides=1, padding='valid', name='conv1')(input)
    x = PReLU(shared_axes=[1,2],name='PReLU1')(x)
    x = MaxPool2D(pool_size=2)(x)
    x = Conv2D(16, (3, 3), strides=1, padding='valid', name='conv2')(x)
    x = PReLU(shared_axes=[1,2],name='PReLU2')(x)
    x = Conv2D(32, (3, 3), strides=1, padding='valid', name='conv3')(x)
    x = PReLU(shared_axes=[1,2],name='PReLU3')(x)
    classifier = Conv2D(2, (1, 1), activation='softmax', name='conv4-1')(x)
    bbox_regress = Conv2D(4, (1, 1), name='conv4-2')(x)
    model = Model([input], [classifier, bbox_regress])
    model.load_weights(weight_path, by_name=True)
    return model

# Load the models

In [5]:
model_12net = create_Kao_Pnet()
model_24net = create_Kao_Rnet()
model_48net = create_Kao_Onet()

# Plot the models

In [6]:
plot_model(model_12net, to_file='model_24net.png')
model_24net.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 24, 24, 3)    0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 22, 22, 28)   784         input_2[0][0]                    
__________________________________________________________________________________________________
prelu1 (PReLU)                  (None, 22, 22, 28)   28          conv1[0][0]                      
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 11, 11, 28)   0           prelu1[0][0]                     
__________________________________________________________________________________________________
conv2 (Con

# Get all the weights in a dictionary

In [7]:
def get_all_weights(src_model):
    original_weights_model = {}
    for idx,layer in enumerate(src_model.layers):
        layer_name = layer.name
        layer_prefix = layer_name[:4]
          
        
        if(layer_prefix == 'conv'):
            weights_biases = []
            ancient_weights = src_model.layers[idx].get_weights()[0]
            ancient_biases =  src_model.layers[idx].get_weights()[1]
            weights_biases.append(ancient_weights)
            weights_biases.append(ancient_biases)
            
            original_weights_model[layer_name] = weights_biases
            
    return original_weights_model

In [27]:
list_original_weights = get_all_weights(model_24net)
print(list_original_weights)

{'conv2': [array([[[[ 0.05701504, -0.03690137,  0.15816815, ...,  0.06590758,
          -0.04544935,  0.07857952],
         [-0.12680408, -0.05586901, -0.02666688, ..., -0.23477486,
          -0.03709188,  0.01061386],
         [-0.2604744 , -0.04326745, -0.02619026, ...,  0.01690904,
          -0.04857722, -0.02117397],
         ...,
         [ 0.05033272, -0.00401206,  0.05515276, ...,  0.00921478,
          -0.04200978, -0.04389929],
         [ 0.07979717, -0.011242  ,  0.03896414, ..., -0.0079498 ,
           0.07034966,  0.03447517],
         [ 0.13143234,  0.02379059,  0.0120946 , ..., -0.00041626,
          -0.06826574, -0.02442481]],

        [[-0.08085082, -0.0674051 ,  0.03632444, ...,  0.0094844 ,
          -0.06760618,  0.04132318],
         [-0.1888587 , -0.0041889 , -0.09677111, ..., -0.07361503,
          -0.01270983,  0.02836486],
         [-0.24460754, -0.02881394, -0.07834327, ..., -0.00647405,
          -0.09472196, -0.02171184],
         ...,
         [-0.09234451, 

# Replace the 2x2 convolutions with 3x3

In [9]:
def zero_padding(original_weights, VERBOSE=False):
    # get the shape
    f_w, f_h, f_d, f_num =original_weights.shape
    
    if(VERBOSE):
        print("f_w: {}, f__h: {}, f_d: {}, f_num: {} " .format(f_w, f_h, f_d, f_num))
        print("")
        
    # change te order, just for visualization
    weights = np.transpose(original_weights, (3,2,1,0))
    if(VERBOSE):
        print("weights -with axes changed")
        print(weights)
        print("")

    # create the 0 padding, forst for width, then for height 
    # draw on a paper to understand
    zeros_width = np.zeros((f_num,f_d,1,f_w))
    zeros_height = np.zeros((f_num,f_d,3,1))
    
    
    weights_vertical_padded = np.concatenate((weights, zeros_width), axis=2)
    weights_horizontal_padded = np.concatenate((weights_vertical_padded, zeros_height), axis=3)
    
    if(VERBOSE):
        print("padded weights -with axes changed")
        print(weights_horizontal_padded)
        print("")
        
    modified_weights = np.transpose(weights_horizontal_padded, (3,2,1,0))
    
    return modified_weights

In [10]:
def replace_weights(list_original_weights, layer_name, VERBOSE):
    list_modified_weights = list_original_weights
    
    original_weights_biases_layer_2x2 = list_original_weights[layer_name]
    original_weights_layer_2x2 = np.array(original_weights_biases_layer_2x2[0])
    original_biases_layer_2x2 = np.array(original_weights_biases_layer_2x2[1])

    # modify the weights
    modified_weights = zero_padding(original_weights_layer_2x2, VERBOSE)
    print("modified_weights.shape: ",modified_weights.shape)
    # replace them
    list_modified_weights[layer_name][0] = modified_weights
    
    return list_modified_weights

# Inspect the new weights

In [11]:
list_modified_weights = replace_weights(list_original_weights, 'conv3', True)

f_w: 2, f__h: 2, f_d: 48, f_num: 64 

weights -with axes changed
[[[[ 7.67413527e-03  1.18614621e-02]
   [ 2.35048570e-02 -3.24537531e-02]]

  [[ 1.07678875e-01  6.56061247e-02]
   [ 8.12826231e-02  3.83714475e-02]]

  [[ 6.14400022e-02 -3.27950567e-02]
   [ 7.42387772e-02 -5.00923907e-03]]

  ...

  [[-3.88107002e-02 -3.31984484e-03]
   [-1.31329438e-02 -2.64904592e-02]]

  [[ 4.64909524e-02 -3.78652103e-02]
   [ 3.54115106e-02 -3.03741340e-02]]

  [[-6.81134732e-03 -2.17999779e-02]
   [-2.96278317e-02 -2.84305252e-02]]]


 [[[-9.17566847e-03  1.00183031e-02]
   [-1.23317530e-02 -1.73956659e-02]]

  [[-3.42370160e-02 -6.68948293e-02]
   [ 1.82679608e-01 -3.71292146e-04]]

  [[ 6.96590617e-02  8.78237039e-02]
   [-1.08418278e-01 -1.18950576e-01]]

  ...

  [[ 2.07620356e-02 -3.56282070e-02]
   [-5.29923290e-03  1.17898369e-02]]

  [[ 8.22932422e-02  4.29809056e-02]
   [-1.24643087e-01 -7.26470053e-02]]

  [[ 1.89319234e-02  2.23363675e-02]
   [-8.95499252e-03 -7.18275830e-03]]]


 [[[-

# Create a new model

In [25]:
def create_A_New_Kao_Rnet (list_modified_weights):
    input = Input(shape=[24, 24, 3])  # change this shape to [None,None,3] to enable arbitraty shape input
    x = Conv2D(28, (3, 3), strides=1, padding='valid', name='conv1')(input)
    x = PReLU(shared_axes=[1, 2], name='prelu1')(x)
    x = MaxPool2D(pool_size=3,strides=2, padding='same')(x)

    x = Conv2D(48, (3, 3), strides=1, padding='valid', name='conv2')(x)
    x = PReLU(shared_axes=[1, 2], name='prelu2')(x)
    x = MaxPool2D(pool_size=3, strides=2)(x)

    x = Conv2D(64, (3, 3), strides=1, padding='valid', name='conv3')(x)
    x = PReLU(shared_axes=[1, 2], name='prelu3')(x)
    x = Permute((3, 2, 1))(x)
    x = Flatten()(x)
    x = Dense(128, name='conv4')(x)
    x = PReLU( name='prelu4')(x)
    classifier = Dense(2, activation='softmax', name='conv5-1')(x)
    bbox_regress = Dense(4, name='conv5-2')(x)
    model = Model([input], [classifier, bbox_regress])
    
    for idx,layer in enumerate(model.layers):
        layer_name = layer.name
        layer_prefix = layer_name[:4]
          
        
        if(layer_prefix == 'conv'):
            print(layer_name)
            
            # set the weights
            weights_biases = []
            weights_biases.append(list_modified_weights[layer_name][0])
            weights_biases.append(list_modified_weights[layer_name][1])
            
            model.layers[idx].set_weights(weights_biases)
            print("after")
            print(model.layers[idx].get_weights())
            
            #model.layers[idx].set_weights(list_modified_weights[layer_name])
    #return model

In [26]:
create_A_New_Kao_Rnet (list_modified_weights)

conv1
[array([[[[ 5.38562238e-03, -1.90136880e-02, -1.09221697e-01,
          -1.52136385e-03,  9.03288126e-02,  1.16114333e-01,
          -7.67034292e-03, -7.80613050e-02, -6.91474900e-02,
          -1.21268481e-02,  4.20494974e-02,  1.38690904e-01,
           1.05301216e-01,  4.16060984e-02,  4.25408334e-02,
           3.44170183e-02,  1.23559460e-01,  1.40705571e-01,
          -6.02744147e-02,  7.72004426e-02, -6.26925528e-02,
           4.03782874e-02, -7.95187056e-03, -1.05846018e-01,
          -5.64281568e-02, -3.84329185e-02, -1.13035224e-01,
          -2.37935781e-03],
         [-1.21674962e-01, -5.41072562e-02, -1.23366073e-01,
          -7.54353479e-02,  4.88145351e-02,  5.45148551e-02,
          -1.33653730e-01,  3.05694044e-02,  2.15419233e-02,
           5.86584359e-02, -5.95475212e-02, -6.76107481e-02,
           2.59799659e-02,  1.37654915e-01,  1.04840443e-01,
          -3.61720473e-02,  9.83004272e-02, -5.49603999e-03,
           7.82467425e-03,  1.32064477e-01,  6.809

after
[array([[[[-5.03155231e-01, -5.88767156e-02, -3.59087646e-01,
           3.81227344e-01, -4.38767560e-02, -8.57151002e-02,
           1.05346948e-01, -2.62339145e-01,  2.57920593e-01,
          -3.16762865e-01,  1.20393112e-01, -1.78890541e-01,
           3.20659205e-02,  5.48509993e-02,  2.38103688e-01,
          -4.49427277e-01, -1.51812620e-02, -6.38561785e-01,
          -1.75464481e-01,  5.57903528e-01,  3.47485989e-01,
           4.02538985e-01, -3.15165877e-01,  1.48909077e-01,
           5.24090081e-02,  3.25124294e-01, -5.24420664e-02,
           3.63276094e-01],
         [-3.99093837e-01, -1.52053669e-01, -2.92599499e-01,
           3.42855483e-01,  2.94439793e-01, -1.50511816e-01,
          -4.31311391e-02, -2.26369306e-01,  2.32669666e-01,
           2.18507156e-01,  5.09582013e-02,  3.80377352e-01,
          -1.75267473e-01,  1.43435776e-01,  2.08610088e-01,
          -1.91553742e-01, -4.89727892e-02, -3.36341798e-01,
           5.65649122e-02, -2.27484167e-01,  2.257

[array([[ 0.07886848, -0.03981328, -0.00228402, ...,  0.06788078,
         0.00597367, -0.11224672],
       [-0.04012114, -0.12219197, -0.07054096, ..., -0.08333632,
         0.03235507,  0.0350475 ],
       [-0.0691801 ,  0.05188036, -0.12339994, ..., -0.08632323,
         0.01723844,  0.11155769],
       ...,
       [-0.06051412, -0.0494619 , -0.04391551, ...,  0.06230614,
        -0.06665421, -0.12225747],
       [ 0.05894375,  0.03508356, -0.10920793, ..., -0.10945842,
        -0.02008811,  0.12142041],
       [-0.12204713, -0.08466065, -0.05980027, ..., -0.01821214,
         0.0800229 , -0.03136873]], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

ValueError: Layer weight shape (256, 128) not compatible with provided weight shape (576, 128)